<a href="https://colab.research.google.com/github/talhaanwarch/OffenseEval2020/blob/master/submissions/Danish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install focal-loss
!pip install keras-tcn==2.8.3

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import lib

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential,Model
from keras.layers import Embedding,CuDNNGRU,CuDNNLSTM,Dense,Dropout,Bidirectional,BatchNormalization,GlobalMaxPooling1D,GlobalAveragePooling1D, MaxPooling1D,SpatialDropout1D,Input,Activation,concatenate,Conv1D
from keras.optimizers import RMSprop,Adam
from keras.initializers import Constant
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,f1_score
import keras
import numpy as np
from focal_loss import BinaryFocalLoss
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


Using TensorFlow backend.


# Load data

In [4]:
cd /content/drive/My Drive/dataset/OffenseEval2020/data/Danish/

/content/drive/My Drive/dataset/OffenseEval2020/data/Danish


In [5]:
ls

cc.da.300.vec     offenseval-da-test-v1-nolabels.tsv
cc.da.300.vec.gz  offenseval-da-training-v1.tsv


In [6]:
train=pd.read_csv( 'offenseval-da-training-v1.tsv',sep="\t")
train.drop(train.tail(n=1).index,inplace=True)
train["tweet"]= train["tweet"].str.replace('@USER', "") 
train['tweet']=train['tweet'].str.replace('\d+', '')
train['tweet']=train['tweet'].str.replace('URL', '')
train["tweet"]= train["tweet"].str.lower()
train=train.sample(frac=1)
train.head()

,id,tweet,subtask_a
2670,3345,det er godt nok thomas,NOT
754,500,nej de kan rejse hjem for nogle af alle de pen...,NOT
1558,363,hvor ser du at jeg forsvare kommunisme? det gø...,OFF
1582,2977,man(d) får associationer til år,NOT
2682,2381,danmark stormer frem i det store udland hvor e...,NOT


In [7]:
test=pd.read_csv( 'offenseval-da-test-v1-nolabels.tsv',sep='\t')
test["tweet"]= test["tweet   subtask_a"].str.replace('@USER', "") 
test['tweet']=test['tweet'].str.replace('\d+', '')
test['tweet']=test['tweet'].str.replace('URL', '')
test['tweet']=test['tweet'].str.lower()
ids=test['id']
test=test['tweet']
test.head()

0    der er syriske "flygtninge" som rejser til ira...
1                               danmark = vitryssland?
2    ja tvangsfjernelser af børn på urigtige oplysn...
3    han kan ikke svensk og forventer et job. hvis ...
4                                    ned med svensken!
Name: tweet, dtype: object

#Label Encoding

In [0]:
labels=train['subtask_a']
train=train['tweet']

In [0]:
le=LabelEncoder()
labels=le.fit_transform(labels)

# Tokenize data

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
max_words = 10000 #frequency of words to be kept
max_len = 200

tokenize = Tokenizer(num_words=max_words)
tokenize.fit_on_texts(train)
sequences = tokenize.texts_to_sequences(train)
word_index = tokenize.word_index
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [11]:
len(sequences_matrix),len(labels)

(2960, 2960)

In [12]:
num_words = min(max_words, len(word_index)) + 1
print(num_words)

9802


In [0]:
test_sequences = tokenize.texts_to_sequences(test)
X_test_sequences = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [14]:
len(X_test_sequences)

329

# Embeddings

In [0]:
embed_size=300
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')

def build_matrix(embedding_path, word_index):
    embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

    nb_words = min(max_words, len(word_index))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    for word, i in word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [16]:
embeddings=build_matrix('cc.da.300.vec', word_index)
embeddings.shape

(9802, 300)

# Base Models

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(train,labels,test_size=0.10,stratify=labels,shuffle=True,random_state=2020)

# Count Vectorizer

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
count_vectorizer.fit(x_train)
X_train_cv = count_vectorizer.transform(x_train)
X_val_cv  = count_vectorizer.transform(x_val)
X_test_cv=  count_vectorizer.transform(test)

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
def logistic_param_selection(X, y, nfolds):
    C= [0.1, 1,3,5,8, 10,12,15]
    param_grid = {'C': C}
    grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=nfolds,scoring='f1_macro')
    grid_search.fit(X, y)
    print(grid_search.best_params_)
    return grid_search.best_score_
count_vectorizer.fit(train)
train_cv = count_vectorizer.transform(train)
logistic_param_selection(train_cv,labels,5)

{'C': 12}


0.7197259792344877

In [20]:
from sklearn.linear_model import LogisticRegression
cv_classifier = LogisticRegression(solver='lbfgs',C=10,max_iter=500,class_weight='balance')
cv_classifier.fit(X_train_cv, y_train)
y_pred = cv_classifier.predict(X_val_cv)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       258
           1       0.71      0.39      0.51        38

    accuracy                           0.90       296
   macro avg       0.82      0.69      0.73       296
weighted avg       0.89      0.90      0.89       296



## TF IDF word vectorizer

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1, 4),
    max_features=20000)
word_vectorizer.fit(x_train)
train_word_features = word_vectorizer.transform(x_train)
test_word_features = word_vectorizer.transform(x_val)


In [22]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',max_iter=500)
classifier.fit(train_word_features, y_train)
y_pred = classifier.predict(test_word_features)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       258
           1       1.00      0.05      0.10        38

    accuracy                           0.88       296
   macro avg       0.94      0.53      0.52       296
weighted avg       0.89      0.88      0.83       296



## TF IDF char vectorizer

In [0]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 6),
    max_features=30000)
char_vectorizer.fit(x_train)
train_char_features = char_vectorizer.transform(x_train)
test_char_features = char_vectorizer.transform(x_val)


In [24]:
classifier = LogisticRegression(solver='lbfgs',max_iter=500)
classifier.fit(train_char_features, y_train)
y_pred = classifier.predict(test_char_features)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       258
           1       1.00      0.05      0.10        38

    accuracy                           0.88       296
   macro avg       0.94      0.53      0.52       296
weighted avg       0.89      0.88      0.83       296



# Models

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val=train_test_split(sequences_matrix,labels,test_size=0.10,stratify=labels,shuffle=True,random_state=2020)

##GRU

In [0]:
def model_gru():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x, x_h, x_c = Bidirectional(CuDNNGRU(50, return_sequences=True, return_state = True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  conc = concatenate([avg_pool, x_h, max_pool])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=Adam(0.001), metrics=['accuracy'])
  return model

## GRU attention

In [0]:
#https://github.com/zake7749/DeepToxic
from keras.layers import Layer,Lambda
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [0]:
def model_gru_attn():
  inp = Input(shape=(max_len,))
  x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  x = SpatialDropout1D(0.1)(x)
  x = Bidirectional(CuDNNLSTM(50, return_sequences=True))(x)
  x = Bidirectional(CuDNNGRU(50, return_sequences=True))(x)
  avg_pool = GlobalAveragePooling1D()(x)
  max_pool = GlobalMaxPooling1D()(x)
  last = Lambda(lambda t: t[:, -1])(x)
  attn = AttentionWeightedAverage()(x)
  conc = concatenate([avg_pool,  max_pool,last,attn])
  outp = Dense(1, activation="sigmoid")(conc)    
  model = Model(inputs=inp, outputs=outp)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=Adam(0.001), metrics=['accuracy'])
  return model

##TCN

In [0]:
from tcn import TCN

def wave_net_activation(x):
    # type: (Layer) -> Layer
    """This method defines the activation used for WaveNet
    described in https://deepmind.com/blog/wavenet-generative-model-raw-audio/
    Args:
        x: The layer we want to apply the activation to
    Returns:
        A new layer with the wavenet activation applied
    """
    tanh_out = Activation('tanh')(x)
    sigm_out = Activation('sigmoid')(x)
    return keras.layers.multiply([tanh_out, sigm_out])

def model_tcn(embedding_matrix):
    
    inp = Input(shape=(max_len,))
    x = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True ,name = 'tnc1')(x)
    x = wave_net_activation(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, name = 'tnc2')(x)
    x = wave_net_activation(x)
    #x = TCN(32,dilations = [1, 2, 4], return_sequences=True, activation = 'wavenet',name = 'tnc3')(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(64, activation="relu")(conc)
    conc = Dense(32, activation="relu")(conc)

    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=RMSprop(), metrics=['accuracy'])
    
    return model

In [0]:
import numpy as np
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)
class_weights=dict(enumerate(class_weights))

## VDCNN

In [0]:
#https://github.com/mukesh-mehta/VDCNN/blob/master/toxic.ipynb
def ConvolutionalBlock(input_shape, num_filters):
    model=Sequential()

    #1st conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same", input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    #2nd conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    return model

#https://www.tensorflow.org/api_docs/python/tf/nn/top_k
def top_kmax(x):
    x=tf.transpose(x, [0, 2, 1])
    k_max = tf.nn.top_k(x, k=top_k)
    return tf.reshape(k_max[0], (-1, num_filters[-1]*top_k))

def conv_shape(conv):
    return conv.get_shape().as_list()[1:]


In [0]:
def model_vdcnn(num_filters, num_classes, sequence_max_length, num_chars, embedding_size, top_k, learning_rate=0.001):
    
    inputs=Input(shape=(sequence_max_length, ), dtype='int32', name='input')
    
    embedded_seq = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inputs)
    embedded_seq = BatchNormalization()(embedded_seq)
    #1st Layer
    conv = Conv1D(filters=64, kernel_size=3, strides=2, padding="same")(embedded_seq)
    
    #ConvBlocks
    for i in range(len(num_filters)):
        conv = ConvolutionalBlock(conv_shape(conv), num_filters[i])(conv)
        conv = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv)
        
    def _top_k(x):
        x = tf.transpose(x, [0, 2, 1])
        k_max = tf.nn.top_k(x, k=top_k)
        return tf.reshape(k_max[0], (-1, num_filters[-1] * top_k))
    
    k_max = Lambda(_top_k, output_shape=(num_filters[-1] * top_k,))(conv)
    
    #fully connected layers
    # in original paper they didn't used dropouts
    fc1=Dense(100, activation='relu', kernel_initializer='he_normal')(k_max)
    fc1=Dropout(0.3)(fc1)
    fc2=Dense(20, activation='relu', kernel_initializer='he_normal')(fc1)
    fc2=Dropout(0.3)(fc2)
    out=Dense(1, activation='sigmoid')(fc2)
    
    
    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer='rmsprop', loss=BinaryFocalLoss(gamma=2), metrics=['accuracy'])
    
    return model

In [0]:
num_filters = [64, 128, 256, 512]
vdcnn_model=model_vdcnn(num_filters=num_filters, num_classes=1,num_chars=69, sequence_max_length=max_len,embedding_size=16,top_k=3)

##KIM

In [0]:
conv_filters=128
def model_kim():
  inp = Input(shape=(max_len,))
  emb = Embedding(num_words,embed_size,embeddings_initializer=Constant(embeddings),input_length=max_len,trainable=False)(inp)
  # Specify each convolution layer and their kernel siz i.e. n-grams 
  conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(emb)
  btch1_1 = BatchNormalization()(conv1_1)
  drp1_1  = Dropout(0.2)(btch1_1)
  actv1_1 = Activation('relu')(drp1_1)
  glmp1_1 = GlobalMaxPooling1D()(actv1_1)

  conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(emb)
  btch1_2 = BatchNormalization()(conv1_2)
  drp1_2  = Dropout(0.2)(btch1_2)
  actv1_2 = Activation('relu')(drp1_2)
  glmp1_2 = GlobalMaxPooling1D()(actv1_2)

  conv1_3 = Conv1D(filters=conv_filters, kernel_size=5)(emb)
  btch1_3 = BatchNormalization()(conv1_3)
  drp1_3  = Dropout(0.2)(btch1_3)
  actv1_3 = Activation('relu')(drp1_3)
  glmp1_3 = GlobalMaxPooling1D()(actv1_3)

  conv1_4 = Conv1D(filters=conv_filters, kernel_size=6)(emb)
  btch1_4 = BatchNormalization()(conv1_4)
  drp1_4  = Dropout(0.2)(btch1_4)
  actv1_4 = Activation('relu')(drp1_4)
  glmp1_4 = GlobalMaxPooling1D()(actv1_4)

  # Gather all convolution layers
  cnct = concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
  drp1 = Dropout(0.2)(cnct)

  dns1  = Dense(32, activation='relu')(drp1)
  btch1 = BatchNormalization()(dns1)
  drp2  = Dropout(0.2)(btch1)

  out = Dense(1, activation='sigmoid')(drp2)   
  model = Model(inputs=inp, outputs=out)
  model.compile(loss=BinaryFocalLoss(gamma=2), optimizer=Adam(0.001), metrics=['accuracy'])
  return model


# Train model

##GRU

In [50]:
gru_model=model_gru()
gru_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 2664 samples, validate on 296 samples
Epoch 1/10
 - 9s - loss: 0.1113 - acc: 0.8697 - val_loss: 0.1032 - val_acc: 0.8716
Epoch 2/10
 - 8s - loss: 0.1028 - acc: 0.8686 - val_loss: 0.0991 - val_acc: 0.8716
Epoch 3/10
 - 8s - loss: 0.0927 - acc: 0.8773 - val_loss: 0.0937 - val_acc: 0.8784
Epoch 4/10
 - 8s - loss: 0.0831 - acc: 0.8851 - val_loss: 0.0937 - val_acc: 0.8750
Epoch 5/10
 - 8s - loss: 0.0749 - acc: 0.9054 - val_loss: 0.0978 - val_acc: 0.8818
Epoch 6/10
 - 8s - loss: 0.0717 - acc: 0.9065 - val_loss: 0.0953 - val_acc: 0.8818
Epoch 7/10
 - 8s - loss: 0.0683 - acc: 0.9114 - val_loss: 0.1023 - val_acc: 0.8851
Epoch 8/10
 - 8s - loss: 0.0654 - acc: 0.9182 - val_loss: 0.0977 - val_acc: 0.8750
Epoch 9/10
 - 8s - loss: 0.0627 - acc: 0.9137 - val_loss: 0.1051 - val_acc: 0.8615
Epoch 10/10
 - 8s - loss: 0.0582 - acc: 0.9242 - val_loss: 0.1117 - val_acc: 0.8716


## GRU attention

In [35]:
attn_gru_model=model_gru_attn()
attn_gru_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 2664 samples, validate on 296 samples
Epoch 1/10
 - 9s - loss: 0.1158 - acc: 0.8690 - val_loss: 0.1033 - val_acc: 0.8615
Epoch 2/10
 - 8s - loss: 0.1026 - acc: 0.8705 - val_loss: 0.1009 - val_acc: 0.8716
Epoch 3/10
 - 8s - loss: 0.0934 - acc: 0.8727 - val_loss: 0.0984 - val_acc: 0.8682
Epoch 4/10
 - 8s - loss: 0.0860 - acc: 0.8836 - val_loss: 0.0954 - val_acc: 0.8682
Epoch 5/10
 - 8s - loss: 0.0786 - acc: 0.8908 - val_loss: 0.0950 - val_acc: 0.8784
Epoch 6/10
 - 8s - loss: 0.0735 - acc: 0.9005 - val_loss: 0.0946 - val_acc: 0.8784
Epoch 7/10
 - 8s - loss: 0.0678 - acc: 0.9118 - val_loss: 0.0972 - val_acc: 0.8750
Epoch 8/10
 - 8s - loss: 0.0647 - acc: 0.9140 - val_loss: 0.0982 - val_acc: 0.8818
Epoch 9/10
 - 8s - loss: 0.0616 - acc: 0.9174 - val_loss: 0.1017 - val_acc: 0.8784
Epoch 10/10
 - 8s - loss: 0.0545 - acc: 0.9242 - val_loss: 0.1188 - val_acc: 0.8818


##TCN

In [36]:
tcn_model=model_tcn(embeddings)
tcn_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 2664 samples, validate on 296 samples
Epoch 1/10
 - 8s - loss: 0.1133 - acc: 0.8619 - val_loss: 0.1066 - val_acc: 0.8716
Epoch 2/10
 - 2s - loss: 0.1073 - acc: 0.8690 - val_loss: 0.1034 - val_acc: 0.8716
Epoch 3/10
 - 2s - loss: 0.1004 - acc: 0.8705 - val_loss: 0.0942 - val_acc: 0.8716
Epoch 4/10
 - 2s - loss: 0.0893 - acc: 0.8765 - val_loss: 0.0891 - val_acc: 0.8716
Epoch 5/10
 - 2s - loss: 0.0811 - acc: 0.8874 - val_loss: 0.0828 - val_acc: 0.8784
Epoch 6/10
 - 2s - loss: 0.0713 - acc: 0.9110 - val_loss: 0.0943 - val_acc: 0.9020
Epoch 7/10
 - 2s - loss: 0.0638 - acc: 0.9208 - val_loss: 0.0962 - val_acc: 0.9088
Epoch 8/10
 - 2s - loss: 0.0582 - acc: 0.9313 - val_loss: 0.0991 - val_acc: 0.9054
Epoch 9/10
 - 2s - loss: 0.0525 - acc: 0.9381 - val_loss: 0.0888 - val_acc: 0.8919
Epoch 10/10
 - 2s - loss: 0.0466 - acc: 0.9482 - val_loss: 0.1021 - val_acc: 0.8851


##VDCNN

In [37]:
vdcnn_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 2664 samples, validate on 296 samples
Epoch 1/10
 - 7s - loss: 0.3589 - acc: 0.7774 - val_loss: 0.1094 - val_acc: 0.8716
Epoch 2/10
 - 3s - loss: 0.1555 - acc: 0.8529 - val_loss: 0.1544 - val_acc: 0.8716
Epoch 3/10
 - 3s - loss: 0.1380 - acc: 0.8634 - val_loss: 0.4015 - val_acc: 0.8716
Epoch 4/10
 - 3s - loss: 0.1304 - acc: 0.8690 - val_loss: 0.1124 - val_acc: 0.8716
Epoch 5/10
 - 3s - loss: 0.1207 - acc: 0.8682 - val_loss: 0.1295 - val_acc: 0.8716
Epoch 6/10
 - 3s - loss: 0.1165 - acc: 0.8697 - val_loss: 0.1148 - val_acc: 0.8716
Epoch 7/10
 - 3s - loss: 0.1063 - acc: 0.8679 - val_loss: 0.1371 - val_acc: 0.8615
Epoch 8/10
 - 3s - loss: 0.0979 - acc: 0.8784 - val_loss: 0.1318 - val_acc: 0.8716
Epoch 9/10
 - 3s - loss: 0.0875 - acc: 0.8833 - val_loss: 0.1332 - val_acc: 0.8547
Epoch 10/10
 - 3s - loss: 0.0747 - acc: 0.9028 - val_loss: 0.1853 - val_acc: 0.8953


##KIM

In [55]:
kim_model=model_kim()
kim_model.fit(x_train,y_train,batch_size=64,epochs=10,verbose=2,class_weight=class_weights,validation_data=(x_val,y_val))

Train on 2664 samples, validate on 296 samples
Epoch 1/10
 - 4s - loss: 0.2791 - acc: 0.5728 - val_loss: 0.1482 - val_acc: 0.7061
Epoch 2/10
 - 2s - loss: 0.1577 - acc: 0.7346 - val_loss: 0.1283 - val_acc: 0.8074
Epoch 3/10
 - 2s - loss: 0.1227 - acc: 0.8131 - val_loss: 0.1005 - val_acc: 0.8649
Epoch 4/10
 - 2s - loss: 0.1089 - acc: 0.8382 - val_loss: 0.1347 - val_acc: 0.8514
Epoch 5/10
 - 2s - loss: 0.0892 - acc: 0.8788 - val_loss: 0.0949 - val_acc: 0.8851
Epoch 6/10
 - 2s - loss: 0.0772 - acc: 0.8998 - val_loss: 0.0915 - val_acc: 0.8649
Epoch 7/10
 - 2s - loss: 0.0670 - acc: 0.9182 - val_loss: 0.0821 - val_acc: 0.8581
Epoch 8/10
 - 2s - loss: 0.0534 - acc: 0.9381 - val_loss: 0.0885 - val_acc: 0.8750
Epoch 9/10
 - 2s - loss: 0.0449 - acc: 0.9396 - val_loss: 0.0985 - val_acc: 0.8986
Epoch 10/10
 - 2s - loss: 0.0410 - acc: 0.9482 - val_loss: 0.1004 - val_acc: 0.8953


# Validate Models

##GRU

In [51]:
y_pred = gru_model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

296/296 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.92      0.93      0.93       258
           1       0.50      0.47      0.49        38

    accuracy                           0.87       296
   macro avg       0.71      0.70      0.71       296
weighted avg       0.87      0.87      0.87       296



## GRU attention

In [39]:
y_pred = attn_gru_model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

296/296 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       0.90      0.97      0.93       258
           1       0.59      0.26      0.36        38

    accuracy                           0.88       296
   macro avg       0.74      0.62      0.65       296
weighted avg       0.86      0.88      0.86       296



##TCN

In [40]:
y_pred = tcn_model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

296/296 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       258
           1       0.56      0.53      0.54        38

    accuracy                           0.89       296
   macro avg       0.74      0.73      0.74       296
weighted avg       0.88      0.89      0.88       296



## VDCNN

In [41]:
y_pred = vdcnn_model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

296/296 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       258
           1       0.68      0.34      0.46        38

    accuracy                           0.90       296
   macro avg       0.80      0.66      0.70       296
weighted avg       0.88      0.90      0.88       296



##KIM

In [56]:
y_pred = kim_model.predict(x_val, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred = (y_pred > 0.5)

print(classification_report(y_val, y_pred))

296/296 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.90      0.99      0.94       258
           1       0.82      0.24      0.37        38

    accuracy                           0.90       296
   macro avg       0.86      0.61      0.66       296
weighted avg       0.89      0.90      0.87       296



# Submit file

In [42]:
y_test_cv = cv_classifier.predict(X_test_cv)
y_test_gru = gru_model.predict(X_test_sequences, batch_size=64, verbose=1)
y_test_gru_atten = attn_gru_model.predict(X_test_sequences, batch_size=64, verbose=1)
y_test_tcn = tcn_model.predict(X_test_sequences, batch_size=64, verbose=1)
# # 

329/329 [==============================] - 0s 1ms/step


In [43]:
y_test=np.mean((y_test_cv.ravel(),y_test_gru.ravel(),y_test_gru_atten.ravel(),y_test_tcn.ravel()),axis=0)
y_test.shape

(329,)

In [44]:
y_test[0:10]

array([0.12462883, 0.18853208, 0.16026854, 0.11188069, 0.15853215,
       0.10720116, 0.24298399, 0.08039539, 0.18485107, 0.15721201])

In [0]:
y_test[y_test>0.5]=1
y_test[y_test<0.5]=0
y_test=y_test.astype('int16').ravel()

y_test=le.inverse_transform(y_test)
y_test=pd.DataFrame(y_test,columns=['label'])
y_test=pd.concat([ids, y_test['label']], axis=1)


y_test.to_csv('/content/weighted.csv',index=False,header=None)


# Reset

In [0]:
import keras
keras.backend.clear_session()